In [35]:
import requests
import pandas as pd

In [26]:
annot = []
flag = 1
while flag:
    search_params = dict(
        sort="updated",
        limit=100,
        search_after=annot[-1]["updated"] if annot else "",
        uri="https://lopentu.github.io/casa/threads-sample-every50.html"
    )

    resp = requests.get("https://api.hypothes.is/api/search", params=search_params)
    if resp.status_code == 200:
        annot_data = resp.json()
        rows = annot_data.get("rows", [])
        flag = len(rows)
        annot.extend(rows)
        print("annot counts: ", len(annot))        
    else:
        print(resp)    

annot counts:  100
annot counts:  200
annot counts:  300
annot counts:  338
annot counts:  338


In [28]:
annot[0]

{'id': 'EzIFsnauEeuSUpfUSvfWww',
 'created': '2021-02-24T14:39:18.309138+00:00',
 'updated': '2021-02-24T14:39:18.309138+00:00',
 'user': 'acct:amy0118@hypothes.is',
 'uri': 'https://lopentu.github.io/casa/threads-sample-every50.html',
 'text': 'comparative',
 'tags': [],
 'group': '__world__',
 'permissions': {'read': ['group:__world__'],
  'admin': ['acct:amy0118@hypothes.is'],
  'update': ['acct:amy0118@hypothes.is'],
  'delete': ['acct:amy0118@hypothes.is']},
 'target': [{'source': 'https://lopentu.github.io/casa/threads-sample-every50.html',
   'selector': [{'type': 'RangeSelector',
     'endOffset': 21,
     'startOffset': 1,
     'endContainer': '/div[1]/div[371]/ol[1]/li[1]',
     'startContainer': '/div[1]/div[371]/ol[1]/li[1]'},
    {'end': 153721, 'type': 'TextPositionSelector', 'start': 153701},
    {'type': 'TextQuoteSelector',
     'exact': '基地台數多又怎樣我家這邊中華比亞太還爛！',
     'prefix': 't Jun  6 12:31:01 2020\n(無主文)\n   ',
     'suffix': '  \n  \n   台灣有10億人口吧，光中華電信1000萬用戶，'}]}],

In [78]:
def get_quote(entry):
    target = entry.get("target", [])[0]    
    selectors = target.get("selector", [])
    for x in selectors:
        if x["type"] == "TextQuoteSelector":
            return x["exact"]
    return ""

def get_div_id(entry):
    target = entry.get("target", [])[0]    
    selectors = target.get("selector", [])
    for x in selectors:
        if x["type"] == "RangeSelector":
            try:
                return int(x["startContainer"].split("/")[2][4:-1])
            except:
                return -1
    return -1

def get_fields(entry):
    user = entry.get("user", "").split(":")[1].split("@")[0]
    exact = get_quote(entry)
    div_id = get_div_id(entry)
    text = entry.get("text", "")
    return (div_id, user, exact, text)
annot_frame = pd.DataFrame(list(map(get_fields, annot)), columns=["id", "annoter", "quote", "annot_text"])
annot_frame = annot_frame.loc[annot_frame.id>=0, :]\
                .loc[annot_frame.annoter != "seantyh"]\
                .sort_values("id")\
                .reset_index(drop=True)
annot_frame["function"] = annot_frame.annot_text.str.extract("(eval|comp)")
annot_frame["is_not_cht"] = annot_frame.annot_text.str.find("非中華") >= 0

In [79]:
annot_frame

,id,annoter,quote,annot_text,function,is_not_cht
0,4,amy0118,中華訊號是還算可以,evaluative,eval,False
1,4,amy0118,用起來都一樣。之前有人測試過，流量達到 1TB降速,comparative?,comp,False
2,10,amy0118,電信業者,不知道是不是有「電信業者」這類的關鍵字就都會被抓下來？(像這篇其實完全無關，只講到遠傳,NaN,False
3,18,amy0118,中華起碼要5.5萬基地台以上才夠拚，3萬確實有點少。 \n 中華電信用戶這麼多基地台還...,細看可算是evaluative，但整體是comparative (基地台數目、用戶/基地台比...,eval,False
4,25,amy0118,永遠記得韓國都5G了！我們中華電信怎麼不是8G，還在搞4G,evaluative,eval,False
...,...,...,...,...,...,...
325,1117,andreashih,這三家要比的話我覺得亞太OK 如果都在市區更沒問題\n 我平常在台北市區走跳 覺得亞太訊號還行,evaluative (非中華),eval,True
326,1117,andreashih,收訊不錯的話 亞太CP值應該更好,evaluative (非中華),eval,True
327,1119,andreashih,當然選亞太啦~資費便宜，收訊也不錯而且亞太5G是和遠傳合作當然選CP值高的呀,evaluative (非中華),eval,True
328,1119,andreashih,亞太的資費CP值一直都滿好的\n 我的生活圈亞太收訊也不錯,evaluative (非中華),eval,True


In [80]:
annot_frame.function.value_counts(dropna=False)

eval    222
comp     97
NaN      11
Name: function, dtype: int64

In [87]:
annot_frame.pivot_table(index=["annoter"], columns=["function"], values="id", aggfunc=len, dropna=False)

function,comp,eval
annoter,,
NanaYang,47,113
amy0118,34,64
andreashih,16,45


In [81]:
annot_frame.to_csv("../../data/annot-every-50.csv", encoding="UTF-8", index=False)

In [76]:
222/1121, 97/1121

(0.19803746654772525, 0.08652988403211419)

全部1121，其中222(20%)是eval，97(9%)是comparative